In [1]:
from pathlib import Path
import datetime
from pytz import timezone
import pandas as pd
import numpy as np

In [2]:
import pickle

### 利用するDataFrame

今回はawareなDatetimeIndexをインデックスとするDataFrameを利用する．

In [3]:
with open("aware_stock_df.pickle", "rb") as f:
    stock_df = pickle.load(f)
    
stock_df

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-04 00:00:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-04 00:01:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-04 00:02:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-04 00:03:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-04 00:04:00+09:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2020-11-30 23:55:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-30 23:56:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-30 23:57:00+09:00,NaN,NaN,NaN,NaN,NaN


### datetime.datetimeによるインデキシング 

上のデータはdatetimeindexがawareなdatetimeなので，インデキシングに利用するdatetimeもawareでなければならない．`pd.DatetimeIndex`(あるいは他のindexでも)の比較はbooleanのndarrayが返る．

ここで，DatetimeIndexとの比較に利用できるのは，naiveかawareのdatetime.datetimeかpd.TimeStampである．datetime.dateを比較する場合にはpd.TimeStampに変換してあげればよい．

In [4]:
jst_timezone = timezone("Asia/Tokyo")

bool_array = stock_df.index > jst_timezone.localize(datetime.datetime(2020,11,16,9,0,0))
sub_stock_df = stock_df.loc[bool_array]
sub_stock_df

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-16 09:01:00+09:00,2708.0,2711.0,2708.0,2708.0,7800.0
2020-11-16 09:02:00+09:00,2708.0,2708.0,2701.0,2702.0,14900.0
2020-11-16 09:03:00+09:00,2703.0,2712.0,2701.0,2703.0,32600.0
2020-11-16 09:04:00+09:00,2703.0,2706.0,2701.0,2702.0,12700.0
2020-11-16 09:05:00+09:00,2703.0,2706.0,2698.0,2705.0,57400.0
...,...,...,...,...,...
2020-11-30 23:55:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-30 23:56:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-30 23:57:00+09:00,NaN,NaN,NaN,NaN,NaN


### 範囲を指定してインデキシング 

もちろん日を意味するdatetime.dateを指定してもエラーが出る

In [5]:
sub_stock_df = stock_df.loc[datetime.date(2020,12,1)]
sub_stock_df

KeyError: datetime.date(2020, 12, 1)

期間指定する場合は素直にboolインデックスを利用する.

In [6]:
bool_array = (jst_timezone.localize(datetime.datetime(2020,11,16,9,0,0)) <= stock_df.index) & (stock_df.index < jst_timezone.localize(datetime.datetime(2020,11,16,15,0,0)))
sub_stock_df = stock_df.loc[bool_array]
sub_stock_df

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-16 09:00:00+09:00,2714.0,2716.0,2705.0,2710.0,111000.0
2020-11-16 09:01:00+09:00,2708.0,2711.0,2708.0,2708.0,7800.0
2020-11-16 09:02:00+09:00,2708.0,2708.0,2701.0,2702.0,14900.0
2020-11-16 09:03:00+09:00,2703.0,2712.0,2701.0,2703.0,32600.0
2020-11-16 09:04:00+09:00,2703.0,2706.0,2701.0,2702.0,12700.0
...,...,...,...,...,...
2020-11-16 14:55:00+09:00,2715.0,2716.0,2715.0,2715.0,6400.0
2020-11-16 14:56:00+09:00,2716.0,2716.0,2714.0,2716.0,25500.0
2020-11-16 14:57:00+09:00,2715.0,2717.0,2715.0,2715.0,13800.0


複数期間の場合は，普通にbooleanのarrayを論理和(&)すればよい

### 複数の特定のインデックスを取り出す

これはDatetimeIndexに限ったことではないが，pd.DatatetimeIndex.isinメソッドを利用できる．`isin`はTimeStampオブジェクトのリストでなくてdatetime.datetimeのリストでもよい

In [7]:
datetime_list = [jst_timezone.localize(datetime.datetime(2020,11,16,9,0,0)),
                 jst_timezone.localize(datetime.datetime(2020,11,16,10,0,0)),
                 jst_timezone.localize(datetime.datetime(2020,11,16,11,0,0)),
                 jst_timezone.localize(datetime.datetime(2020,11,16,13,0,0)),
                 jst_timezone.localize(datetime.datetime(2020,11,16,14,0,0)),
                ]

bool_array = stock_df.index.isin(datetime_list)
sub_stock_df = stock_df.loc[bool_array]
sub_stock_df

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-16 09:00:00+09:00,2714.0,2716.0,2705.0,2710.0,111000.0
2020-11-16 10:00:00+09:00,2710.0,2714.0,2710.0,2713.0,9400.0
2020-11-16 11:00:00+09:00,2715.0,2716.0,2715.0,2715.0,1100.0
2020-11-16 13:00:00+09:00,2706.0,2706.0,2705.0,2705.0,500.0
2020-11-16 14:00:00+09:00,2717.0,2717.0,2716.0,2716.0,5000.0


### 複数の特定の日時のものを取り出す 

ある日にちもののみ取り出すときに，その日にちが複数ある場合．`pd.DatetimeIndex.date`によって`datetime.date`のndarrayを返し，numpyの`np.isin`か`np.in1d`をとることが考えられる．このとき`datetime.date`になるので，タイムゾーンとかは無視される．<- 実は`np.isin`と`np.in1d`はobject typeのndarrayに関して速度が低下することが分かっている．そのためUNIX時間などに変換するか`pd.DatetimeIndex`の`isin`メソッドを利用するべき

In [8]:
date_list = [datetime.date(2020, 11, 10),
             datetime.date(2020, 11, 11),
             datetime.date(2020, 11, 16),
             datetime.date(2020, 11, 17)
            ]
date_array = np.array(date_list)
bool_array = np.in1d(stock_df.index.date, date_array)

sub_stock_df = stock_df.loc[bool_array]
sub_stock_df

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-10 00:00:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-10 00:01:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-10 00:02:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-10 00:03:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-10 00:04:00+09:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2020-11-17 23:55:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-17 23:56:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-17 23:57:00+09:00,NaN,NaN,NaN,NaN,NaN


ここで，あるpd.DataFrameのdatetimeindexをdateの形式にするには，タイムゾーンを取り除いてnaiveにしたのちdateへ切り捨てを行う．

In [15]:
date_list = [datetime.date(2020, 11, 10),
             datetime.date(2020, 11, 11),
             datetime.date(2020, 11, 16),
             datetime.date(2020, 11, 17)
            ]
date_datetimeindex = pd.DatetimeIndex(date_list)
bool_array = stock_df.index.tz_localize(None).floor("D").isin(date_datetimeindex)

sub_stock_df = stock_df.loc[bool_array]
sub_stock_df

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-10 00:00:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-10 00:01:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-10 00:02:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-10 00:03:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-10 00:04:00+09:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2020-11-17 23:55:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-17 23:56:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-17 23:57:00+09:00,NaN,NaN,NaN,NaN,NaN


### 特定の曜日のデータを取得

pd.DatetimeIndex.weekdayを指定することでブールインデックスを取得できる．0が月曜日で6が日曜日(datetime.datetime.weekdayと同じ)このときpandas.DatetimeIndex.weekdayを取得するのには時間がかかるため，代入しておくなどの注意が必要．

In [13]:
bool_array = stock_df.index.weekday==0
stock_df.loc[bool_array]

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-09 09:00:00+09:00,2711.0,2719.0,2707.0,2710.0,0.0
2020-11-09 09:01:00+09:00,2711.0,2712.0,2701.0,2706.0,12900.0
2020-11-09 09:02:00+09:00,2705.0,2710.0,2701.0,2701.0,9200.0
2020-11-09 09:03:00+09:00,2702.0,2703.0,2692.0,2694.0,19300.0
2020-11-09 09:04:00+09:00,2696.0,2698.0,2686.0,2688.0,10800.0
...,...,...,...,...,...
2020-11-30 23:55:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-30 23:56:00+09:00,NaN,NaN,NaN,NaN,NaN
2020-11-30 23:57:00+09:00,NaN,NaN,NaN,NaN,NaN


### 毎日のある時刻を取得 

#### 一つのデータを取得する場合

datetime.timeオブジェクトをインデックスに指定することで，毎日のその時刻のデータを取得できる．これはdatetimeindexがawareなdatetimeの値でもよい．
注意しなければいけないのは，以下では9時00分00秒のデータを取得していること．これは`datetime.time(9)`としても同じ，もちろんリサンプリング等はされない．

In [14]:
stock_df.loc[datetime.time(9,0)].head(5)

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-04 09:00:00+09:00,2669.0,2670.0,2658.0,2664.0,93000.0
2020-11-05 09:00:00+09:00,2631.0,2641.0,2630.0,2639.0,55000.0
2020-11-06 09:00:00+09:00,2615.0,2644.0,2609.0,2617.0,90700.0
2020-11-09 09:00:00+09:00,2711.0,2719.0,2707.0,2710.0,0.0
2020-11-10 09:00:00+09:00,2780.0,2782.0,2770.0,2777.0,142700.0


実は，これは`pandas.DataFrame.at_time`を適用するのと等価である

In [15]:
stock_df.at_time(datetime.time(9,0)).head(5)

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-04 09:00:00+09:00,2669.0,2670.0,2658.0,2664.0,93000.0
2020-11-05 09:00:00+09:00,2631.0,2641.0,2630.0,2639.0,55000.0
2020-11-06 09:00:00+09:00,2615.0,2644.0,2609.0,2617.0,90700.0
2020-11-09 09:00:00+09:00,2711.0,2719.0,2707.0,2710.0,0.0
2020-11-10 09:00:00+09:00,2780.0,2782.0,2770.0,2777.0,142700.0


**コラム**  
このデータベースではyahoofinanceのapiを利用してデータを修正しているが，データの始まり・終わりが特徴的である．以下では取引時間の終わりと始まりのデータを集めてみる．すると，11時30分以降の1分間のデータが取得できていることが分かる．

In [16]:
print("at  9:00 am:\n",stock_df.at_time(datetime.time(9,0)).head(5))
print("at 11:30 am:\n",stock_df.at_time(datetime.time(11,30)).head(5))
print("at 12:30 am:\n",stock_df.at_time(datetime.time(12,30)).head(5))
print("at 15:00 am:\n",stock_df.at_time(datetime.time(15,0)).head(5))

at  9:00 am:
                            Open_6502  High_6502  Low_6502  Close_6502  \
timestamp                                                               
2020-11-04 09:00:00+09:00     2669.0     2670.0    2658.0      2664.0   
2020-11-05 09:00:00+09:00     2631.0     2641.0    2630.0      2639.0   
2020-11-06 09:00:00+09:00     2615.0     2644.0    2609.0      2617.0   
2020-11-09 09:00:00+09:00     2711.0     2719.0    2707.0      2710.0   
2020-11-10 09:00:00+09:00     2780.0     2782.0    2770.0      2777.0   

                           Volume_6502  
timestamp                               
2020-11-04 09:00:00+09:00      93000.0  
2020-11-05 09:00:00+09:00      55000.0  
2020-11-06 09:00:00+09:00      90700.0  
2020-11-09 09:00:00+09:00          0.0  
2020-11-10 09:00:00+09:00     142700.0  
at 11:30 am:
                            Open_6502  High_6502  Low_6502  Close_6502  \
timestamp                                                               
2020-11-04 11:30:00+09:00  

開始時刻の一分前と終了時刻の一分後は

In [17]:
print("at  8:59 am:\n",stock_df.at_time(datetime.time(8,59)).head(5))
print("at 11:31 am:\n",stock_df.at_time(datetime.time(11,31)).head(5))
print("at 12:29 am:\n",stock_df.at_time(datetime.time(12,29)).head(5))
print("at 15:01 am:\n",stock_df.at_time(datetime.time(15,1)).head(5))

at  8:59 am:
                            Open_6502  High_6502  Low_6502  Close_6502  \
timestamp                                                               
2020-11-04 08:59:00+09:00        NaN        NaN       NaN         NaN   
2020-11-05 08:59:00+09:00        NaN        NaN       NaN         NaN   
2020-11-06 08:59:00+09:00        NaN        NaN       NaN         NaN   
2020-11-10 08:59:00+09:00        NaN        NaN       NaN         NaN   
2020-11-11 08:59:00+09:00        NaN        NaN       NaN         NaN   

                           Volume_6502  
timestamp                               
2020-11-04 08:59:00+09:00          NaN  
2020-11-05 08:59:00+09:00          NaN  
2020-11-06 08:59:00+09:00          NaN  
2020-11-10 08:59:00+09:00          NaN  
2020-11-11 08:59:00+09:00          NaN  
at 11:31 am:
                            Open_6502  High_6502  Low_6502  Close_6502  \
timestamp                                                               
2020-11-04 11:31:00+09:00  

ここから，yahoofinanceで得られるデータについて，開始時刻はぴったり始まり，一分前のデータを取得できない．しかし終了時刻については，15時00分はぴったり終了し，一分後のデータが取れないが，11時30分の一分間はデータが取得されることが分かる．

#### 範囲指定 

datetime.timeをスライスのように利用することで，毎日のある時刻から時刻までを取得できる．しかし，これは通常のスライスと異なり，最後を含んでしまうことに注意する．これはデータを少しづつとるときにデータの重複を起こしてしまう．

In [18]:
stock_df.loc[datetime.time(9,0):datetime.time(11,30)]

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-04 09:00:00+09:00,2669.0,2670.0,2658.0,2664.0,93000.0
2020-11-04 09:01:00+09:00,2663.0,2664.0,2650.0,2652.0,17600.0
2020-11-04 09:02:00+09:00,2649.0,2655.0,2646.0,2649.0,19200.0
2020-11-04 09:03:00+09:00,2652.0,2670.0,2651.0,2670.0,31200.0
2020-11-04 09:04:00+09:00,2671.0,2674.0,2670.0,2674.0,12800.0
...,...,...,...,...,...
2020-11-30 11:26:00+09:00,2935.0,2936.0,2932.0,2932.0,7600.0
2020-11-30 11:27:00+09:00,2933.0,2934.0,2932.0,2932.0,3600.0
2020-11-30 11:28:00+09:00,2931.0,2933.0,2931.0,2932.0,4600.0


そこで，対応するメソッドである`pandas.Dataframe.between_time`を利用し，`include_end`を`False`と指定することで一般的なスライスと同等にできる．

In [19]:
stock_df.between_time(start_time=datetime.time(9,0), end_time=datetime.time(11,30), include_end=False)

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-04 09:00:00+09:00,2669.0,2670.0,2658.0,2664.0,93000.0
2020-11-04 09:01:00+09:00,2663.0,2664.0,2650.0,2652.0,17600.0
2020-11-04 09:02:00+09:00,2649.0,2655.0,2646.0,2649.0,19200.0
2020-11-04 09:03:00+09:00,2652.0,2670.0,2651.0,2670.0,31200.0
2020-11-04 09:04:00+09:00,2671.0,2674.0,2670.0,2674.0,12800.0
...,...,...,...,...,...
2020-11-30 11:25:00+09:00,2935.0,2936.0,2934.0,2935.0,7600.0
2020-11-30 11:26:00+09:00,2935.0,2936.0,2932.0,2932.0,7600.0
2020-11-30 11:27:00+09:00,2933.0,2934.0,2932.0,2932.0,3600.0


#### 複数の条件を範囲で指定する場合

`pandas.DatetimeIndex.indexer_between_time`を使って対応するDatetimeIndexのインデックスが取得できる．

In [20]:
am_indice = stock_df.index.indexer_between_time(start_time=datetime.time(9,0), end_time=datetime.time(11,30), include_end=False)
am_indice

array([  540,   541,   542, ..., 28767, 28768, 28769], dtype=int64)

これを複数組み合わせることが考えられるが，インデックスを組み合わせた場合，その順序によって最終的に取得できる順番も変わってしまう．そのため，インデックスをそのまま組み合わせた後に，順番を並び替える必要がある．そこで，ブールインデックスとして処理することを考える．

In [21]:
bool_array = np.array([False]*len(stock_df))
am_indice = stock_df.index.indexer_between_time(start_time=datetime.time(9,0), end_time=datetime.time(11,30), include_end=False)
bool_array[am_indice] = True
pm_indice = stock_df.index.indexer_between_time(start_time=datetime.time(12,30), end_time=datetime.time(15,0), include_end=False)
bool_array[pm_indice] = True

intraday_stock_indice = stock_df.index[bool_array]
intraday_stock_indice

DatetimeIndex(['2020-11-04 09:00:00+09:00', '2020-11-04 09:01:00+09:00',
               '2020-11-04 09:02:00+09:00', '2020-11-04 09:03:00+09:00',
               '2020-11-04 09:04:00+09:00', '2020-11-04 09:05:00+09:00',
               '2020-11-04 09:06:00+09:00', '2020-11-04 09:07:00+09:00',
               '2020-11-04 09:08:00+09:00', '2020-11-04 09:09:00+09:00',
               ...
               '2020-11-30 14:50:00+09:00', '2020-11-30 14:51:00+09:00',
               '2020-11-30 14:52:00+09:00', '2020-11-30 14:53:00+09:00',
               '2020-11-30 14:54:00+09:00', '2020-11-30 14:55:00+09:00',
               '2020-11-30 14:56:00+09:00', '2020-11-30 14:57:00+09:00',
               '2020-11-30 14:58:00+09:00', '2020-11-30 14:59:00+09:00'],
              dtype='datetime64[ns, Asia/Tokyo]', name='timestamp', length=6600, freq=None)

In [22]:
intraday_stock_df = stock_df.loc[intraday_stock_indice,:]
intraday_stock_df

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-11-04 09:00:00+09:00,2669.0,2670.0,2658.0,2664.0,93000.0
2020-11-04 09:01:00+09:00,2663.0,2664.0,2650.0,2652.0,17600.0
2020-11-04 09:02:00+09:00,2649.0,2655.0,2646.0,2649.0,19200.0
2020-11-04 09:03:00+09:00,2652.0,2670.0,2651.0,2670.0,31200.0
2020-11-04 09:04:00+09:00,2671.0,2674.0,2670.0,2674.0,12800.0
...,...,...,...,...,...
2020-11-30 14:55:00+09:00,2908.0,2908.0,2905.0,2905.0,26300.0
2020-11-30 14:56:00+09:00,2905.0,2908.0,2905.0,2907.0,21700.0
2020-11-30 14:57:00+09:00,2908.0,2909.0,2906.0,2907.0,27200.0
